In [3]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
os.getcwd()

'/Users/tahirabbas'

In [4]:
os.chdir('/Users/tahirabbas/Desktop/imp')

In [6]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_reports = []
        self.create_block(proof=1, previous_hash='0')
        self.reports = {}

    def create_block(self, proof, previous_hash):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'proof': proof,
            'previous_hash': previous_hash,
            'reports': self.current_reports
        }
        self.chain.append(block)
        self.current_reports = []
        return block

    def add_report(self, stress_value, diagnosis, key):
        self.reports.setdefault(key, []).append({
            'stress_value': stress_value,
            'diagnosis': diagnosis
        })
        self.current_reports.append({
            'stress_value': stress_value,
            'diagnosis': diagnosis
        })

    def get_last_block(self):
        return self.chain[-1]

    def get_reports_by_key(self, key):
        return self.reports.get(key, [])

def load_and_prepare_data():
    df = pd.read_csv('heart.csv')
    label_encoder = LabelEncoder()
    df['Sex'] = label_encoder.fit_transform(df['Sex'])
    df['Blood Pressure'] = df['Blood Pressure'].str.replace('/', '').astype(float)
    df.drop(['Patient ID', 'Diabetes', 'Family History', "Smoking", 'Alcohol Consumption', 'Diet', 'Previous Heart Problems', 'Medication Use', 'Sedentary Hours Per Day', 'Income', 'BMI', 'Triglycerides', 'Country', 'Continent', 'Hemisphere', 'Heart Attack Risk', 'Obesity'], axis=1, inplace=True)
    y = df['Stress Level']
    X = df.drop(["Stress Level"], axis=1)
    return X, y

def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rf_clf = RandomForestClassifier(random_state=42)
    xgb_clf = XGBClassifier(random_state=42)
    voting_clf = VotingClassifier(estimators=[('rf', rf_clf), ('xgb', xgb_clf)], voting='soft')
    voting_clf.fit(X_train_scaled, y_train)
    return voting_clf, scaler

def predict_stress_level(model, scaler, user_input):
    user_input_scaled = scaler.transform([user_input])
    predicted_stress_level = model.predict(user_input_scaled)[0]
    return predicted_stress_level

def get_diagnosis(stress_level):
    if stress_level <= 3:
        return "Stress level is in control."
    elif stress_level <= 7:
        return "Moderate stress level. Recommend stress management techniques."
    else:
        return "Consult a doctor, your stress level is very high."

def get_user_input():
    age = float(input("Enter Age: "))
    sex = input("Enter Sex (Male/Female): ")
    sex = 1 if sex.lower() == 'male' else 0
    cholesterol = float(input("Enter Cholesterol level: "))
    blood_pressure = float(input("Enter Blood Pressure: "))
    heart_rate = float(input("Enter Heart Rate: "))
    exercise_hours_per_week = float(input("Enter Exercise Hours Per Week: "))
    physical_activity_days_per_week = float(input("Enter Physical Activity Days Per Week: "))
    sleep_hours_per_day = float(input("Enter Sleep Hours Per Day: "))
    
    user_input = [age, sex, cholesterol, blood_pressure, heart_rate, exercise_hours_per_week, physical_activity_days_per_week, sleep_hours_per_day]
    return user_input

X, y = load_and_prepare_data()
model, scaler = train_model(X, y)
user_input = get_user_input()
predicted_stress_level = predict_stress_level(model, scaler, user_input)
diagnosis = get_diagnosis(predicted_stress_level)

blockchain = Blockchain()
key = input("Enter a key to save the report: ")
blockchain.add_report(predicted_stress_level, diagnosis, key)
print("Medical report created and saved successfully!")


Enter Age: 67
Enter Sex (Male/Female): 1
Enter Cholesterol level: 208
Enter Blood Pressure: 15888
Enter Heart Rate: 72
Enter Exercise Hours Per Week: 4.16
Enter Physical Activity Days Per Week: 0
Enter Sleep Hours Per Day: 6


/Users/tahirabbas/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Enter a key to save the report: 0110
Medical report created and saved successfully!


In [7]:
def retrieve_reports(blockchain, key):
    reports = blockchain.reports.get(key, None)
    if reports is None:
        return "No reports found for this key."
    else:
        return reports

key_to_retrieve = input("Enter your key to retrieve the reports: ")
reports = retrieve_reports(blockchain, key_to_retrieve)

if isinstance(reports, list):
    print("All medical reports associated with the key:")
    for i, report in enumerate(reports, start=1):
        print(f"Report {i}: Stress Value: {report['stress_value']}, Diagnosis: {report['diagnosis']}")
else:
    print(reports)


Enter your key to retrieve the reports: 0110
All medical reports associated with the key:
Report 1: Stress Value: 6, Diagnosis: Moderate stress level. Recommend stress management techniques.
